In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, get_anni_op_mpo
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import unittest

In [2]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [11]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

e_ground = -67.46923609481743
e_1st_ex = -67.00839085980371
e_2nd_ex = -67.00038986115365

[1, 16, 62, 100, 154, 100, 62, 16, 1]


In [4]:
psi = generate_single_state(8, [3, 3, 3, 3, 3, 0, 0, 0])

en_min, psi_min = ptn.calculate_ground_state_local_twosite(mpo_ref, psi, 10, 25, 0)
e0 = en_min[-1]
print(e0 - e_ground)
print(psi_min.bond_dims)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov.py:40: RuntimeWarning: beta[0] ~= 0 encountered during Lanczos iteration.
  warnings.warn(
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov.py:40: RuntimeWarning: beta[2] ~= 0 encountered during Lanczos iteration.
  warnings.warn(


-1.1368683772161603e-13
[1, 4, 16, 64, 130, 64, 16, 4, 1]


In [5]:
psi_test = copy.deepcopy(psi_min)
psi_test.orthonormalize('right')
psi_test.orthonormalize('left')
psi_test.orthonormalize('right')
psi_test.compress_direct_svd_left_max_bond(0, 150)
psi_test.orthonormalize('left')
psi_test.orthonormalize('right')
psi_test.orthonormalize('left')

print(psi_test.bond_dims)

print(ptn.operation.operator_average(psi_test, mpo_ref) - e_ground)

[1, 4, 16, 64, 130, 64, 16, 4, 1]
(-1.2789769243681803e-13+0j)


In [6]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_dmrg.pkl"
with open(filename, 'wb') as file:
    pickle.dump(psi_min, file)

In [16]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_dmrg.pkl"
with open(filename, 'rb') as file:
    NH3_ground = pickle.load(file)

print(ptn.operation.operator_average(NH3_ground, mpo_ref) - e_ground)

(-1.7337242752546445e-12+0j)


1st excited state:

In [17]:
minus_ground = copy.deepcopy(NH3_ground)
minus_ground.A[0] = -minus_ground.A[0]
psi_ex_initial = ptn.operation.add_mps(generate_single_state(8, [3, 3, 3, 3, 3, 0, 0, 0]), minus_ground)
psi_ex_initial.orthonormalize('right')
psi_ex_initial.orthonormalize('left')

1.0000000000000004

In [19]:
en_1st_ex, psi_1st_ex = ptn.calculate_ground_state_local_twosite(mpo_ref, psi_ex_initial, 5, 25, 0)
e1 = en_1st_ex[-1]
print(e1 - e_1st_ex)
print(psi_1st_ex.bond_dims)

-0.46084523501548347
[1, 4, 16, 64, 130, 64, 16, 4, 1]


In [ ]:
psi_test_1st_ex = copy.deepcopy(psi_1st_ex)
psi_test_1st_ex.orthonormalize('right')
psi_test_1st_ex.orthonormalize('left')
psi_test_1st_ex.orthonormalize('right')
psi_test_1st_ex.compress_direct_svd_left_max_bond(0, 120)
psi_test_1st_ex.orthonormalize('left')
psi_test_1st_ex.orthonormalize('right')
psi_test_1st_ex.orthonormalize('left')

print(psi_test_1st_ex.bond_dims)

print(ptn.operation.operator_average(psi_test_1st_ex, mpo_ref) - e_1st_ex)

In [8]:
# anni_3_up = get_anni_op_mpo(3, 0, 7)
# initial = ptn.operation.apply_operator(anni_3_up, copy.deepcopy(water_ground))

test: (attention, here the index for ED is inversed!)

In [9]:
# norm((ptn.fermi_sim.annihil_op(14, 2**(7)) @ (copy.deepcopy(water_ground)).as_vector()) + initial.as_vector())

In [10]:
# initial.orthonormalize('right')

# filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(initial, file)